In [1]:
import pandas as pd
import numpy as np

### Import csv & make copy

In [2]:
houses = pd.read_csv('../Datasets/train.csv')

In [3]:
houses.shape

(1460, 81)

In [4]:
houses_copy = houses.copy()

In [5]:
houses_copy.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Check for missing values

In [6]:
missing_values = houses_copy.isna().sum().sort_values(ascending = False)

missing_values

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
                 ... 
CentralAir          0
SaleCondition       0
Heating             0
TotalBsmtSF         0
Id                  0
Length: 81, dtype: int64

In [7]:
missing_values_perc = missing_values/len(houses_copy)
missing_values_perc

PoolQC           0.995205
MiscFeature      0.963014
Alley            0.937671
Fence            0.807534
FireplaceQu      0.472603
                   ...   
CentralAir       0.000000
SaleCondition    0.000000
Heating          0.000000
TotalBsmtSF      0.000000
Id               0.000000
Length: 81, dtype: float64

### Look more closely at columns with over 80% missing values to make sure that they can be deleted.

In [8]:
missing_values_perc.loc[missing_values_perc.gt(0.8)].index

Index(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], dtype='object')

Alley: Type of alley access

In [9]:
houses_copy['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

PoolQC: Pool quality

In [10]:
houses_copy['PoolQC'].value_counts()

Gd    3
Fa    2
Ex    2
Name: PoolQC, dtype: int64

Fence: Fence quality

In [11]:
houses_copy['Fence'].value_counts()

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

MiscFeature: Miscellaneous feature not covered in other categories

In [12]:
houses_copy['MiscFeature'].value_counts()

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

MiscVal: $Value of miscellaneous feature.  
  
This column applies to the MiscFeature column, but did not show up as having a high percentage of missing values, so I looked closer. It appears that the 0 values refer mainly to the missing values in MiscFeature, so this column can also be deleted. 

In [13]:
houses_copy['MiscVal'].value_counts()

0        1408
400        11
500         8
700         5
450         4
2000        4
600         4
1200        2
480         2
1150        1
800         1
15500       1
620         1
3500        1
560         1
2500        1
1300        1
1400        1
350         1
8300        1
54          1
Name: MiscVal, dtype: int64

### Drop columns with over 80% missing values & drop MiscVal column

In [14]:
houses_copy = houses_copy.drop(columns=['Alley', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal'])

#### Check that it worked (previously had 81 columns)

In [15]:
houses_copy.shape

(1460, 76)

In [16]:
missing_values2 = houses_copy.isna().sum().sort_values(ascending = False)
missing_values2

FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageQual        81
GarageType        81
                ... 
CentralAir         0
HeatingQC          0
Heating            0
SaleCondition      0
Id                 0
Length: 76, dtype: int64

In [17]:
missing_values_perc2 = missing_values2/len(houses_copy)
missing_values_perc2

FireplaceQu      0.472603
LotFrontage      0.177397
GarageCond       0.055479
GarageQual       0.055479
GarageType       0.055479
                   ...   
CentralAir       0.000000
HeatingQC        0.000000
Heating          0.000000
SaleCondition    0.000000
Id               0.000000
Length: 76, dtype: float64

## Export to csv as cleaned dataset

In [18]:
houses_copy.to_csv('houses_cleaned.csv', sep=',', index=False)

## Check which columns have very little variation

I noticed that some columns appeared to contain very little variation and based on my domain knowledge, did not seem likely to have a significant impact on price. I explored some of these columns and found that the variation within them was very minimal. 

In [19]:
#99.6% of values are 'Pave'
(houses_copy['Street'].value_counts())/len(houses_copy)

Pave    0.99589
Grvl    0.00411
Name: Street, dtype: float64

In [20]:
#99.9% of values are 'AllPub'
(houses_copy['Utilities'].value_counts())/len(houses_copy)

AllPub    0.999315
NoSeWa    0.000685
Name: Utilities, dtype: float64

In [21]:
#98.2% of values are 'CompShg'
(houses_copy['RoofMatl'].value_counts())/len(houses_copy)

CompShg    0.982192
Tar&Grv    0.007534
WdShngl    0.004110
WdShake    0.003425
Metal      0.000685
Roll       0.000685
Membran    0.000685
ClyTile    0.000685
Name: RoofMatl, dtype: float64

Rather than manually going through all 76 remaining columns, I decided to leave these and rather take a subset of columns that had enough variation and, according to my domain knowledge, could plausibly influence housing price. 

## Create subset of values anticipated to have a significant impact on house prices to examine further

I examined the data_description file on Kaggle, which explained what each of the columns was referring to. 

In [22]:
houses_copy.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodD

In [23]:
houses_copy.shape

(1460, 76)

I selected the most obvious variables to include, which I expected were likely to have an influence on sale price:  

- OverallQual: Overall material and finish quality  
- OverallCond: Overall condition rating  
- YearBuilt: Original construction date  
- YearRemodAdd: Remodel date (same as construction date if no remodeling or additions)
- MoSold: Month Sold (MM) - to check for seasonality
- YrSold: Year Sold (YYYY)
- GarageCars: Size of garage in car capacity
- BedroomAbvGr: number of bedrooms 
- LotArea: Lot size in square feet

I then explored a couple of other variables that I thought would also be worthwhile including.

With the assumption that having a pool increases housing value, I wanted to include the presence/size of the pool, but looking closer at that column, I decided that there are too few properties with a pool to draw meaningful conclusions, so I left it out.

In [24]:
#99.5% of values are 0
(houses_copy['PoolArea'].value_counts())/len(houses_copy)

0      0.995205
738    0.000685
648    0.000685
576    0.000685
555    0.000685
519    0.000685
512    0.000685
480    0.000685
Name: PoolArea, dtype: float64

Next, I examined more closely the two bathroom columns: FullBath & Halfbath, and decided to make a new column combining them.

## Add new column 'TotalBath'

In [25]:
houses_copy['FullBath'].value_counts()

2    768
1    650
3     33
0      9
Name: FullBath, dtype: int64

In [26]:
houses_copy['HalfBath'].value_counts()

0    913
1    535
2     12
Name: HalfBath, dtype: int64

First, I converted the HalfBath column to floats, then I created the new column and assigned the HalfBath values as follows:
- 0 --> 0
- 1 --> 0.5
- 2 --> 1  
  
Then I summed the converted HalfBath values and the FullBath values, to see the total number of bathrooms. 

In [27]:
houses_copy[['FullBath', 'HalfBath']]

,FullBath,HalfBath
0,2,1
1,2,0
2,2,1
3,1,0
4,2,1
...,...,...
1455,2,1
1456,2,0
1457,2,0
1458,1,0


In [28]:
houses_copy['HalfBath'] = houses_copy['HalfBath'].astype(float)
houses_copy[['FullBath', 'HalfBath']]

,FullBath,HalfBath
0,2,1.0
1,2,0.0
2,2,1.0
3,1,0.0
4,2,1.0
...,...,...
1455,2,1.0
1456,2,0.0
1457,2,0.0
1458,1,0.0


In [29]:
houses_copy['TotalBath'] = houses_copy['HalfBath'].replace({0: 0, 1: 0.5, 2: 1})

In [30]:
houses_copy[['FullBath', 'HalfBath', 'TotalBath']]

,FullBath,HalfBath,TotalBath
0,2,1.0,0.5
1,2,0.0,0.0
2,2,1.0,0.5
3,1,0.0,0.0
4,2,1.0,0.5
...,...,...,...
1455,2,1.0,0.5
1456,2,0.0,0.0
1457,2,0.0,0.0
1458,1,0.0,0.0


In [31]:
houses_copy['TotalBath'] = houses_copy['FullBath'] + houses_copy['TotalBath'] 

### Check that it worked

In [32]:
houses_copy[['FullBath', 'HalfBath', 'TotalBath']]

,FullBath,HalfBath,TotalBath
0,2,1.0,2.5
1,2,0.0,2.0
2,2,1.0,2.5
3,1,0.0,1.0
4,2,1.0,2.5
...,...,...,...
1455,2,1.0,2.5
1456,2,0.0,2.0
1457,2,0.0,2.0
1458,1,0.0,1.0


In [33]:
houses_copy['TotalBath'].value_counts()

1.0    469
2.0    433
2.5    334
1.5    180
3.0     20
3.5     18
0.0      3
0.5      3
Name: TotalBath, dtype: int64

## Create subset

In [34]:
houses_subset = houses_copy[['SalePrice', 'OverallCond', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MoSold', 'YrSold', 'LotArea', 'TotalBath', 'BedroomAbvGr', 'GarageCars']]

In [35]:
houses_subset.head()

,SalePrice,OverallCond,OverallQual,YearBuilt,YearRemodAdd,MoSold,YrSold,LotArea,TotalBath,BedroomAbvGr,GarageCars
0,208500,5,7,2003,2003,2,2008,8450,2.5,3,2
1,181500,8,6,1976,1976,5,2007,9600,2.0,3,2
2,223500,5,7,2001,2002,9,2008,11250,2.5,3,2
3,140000,5,7,1915,1970,2,2006,9550,1.0,3,3
4,250000,5,8,2000,2000,12,2008,14260,2.5,4,3


In [36]:
houses_subset.dtypes

SalePrice         int64
OverallCond       int64
OverallQual       int64
YearBuilt         int64
YearRemodAdd      int64
MoSold            int64
YrSold            int64
LotArea           int64
TotalBath       float64
BedroomAbvGr      int64
GarageCars        int64
dtype: object

In [37]:
houses_subset.shape

(1460, 11)

## Export to csv

In [38]:
houses_subset.to_csv('houses_subset.csv', sep=',', index=False)

## Create second subset

After running analyses on the above subset, I wanted to see if I could find stronger correlations between my depedent variable and my independent variables, so I decided to go back to my original dataset, check correlations and make a new data subset based on that.

In [39]:
houses_copy.corr()['SalePrice'].sort_values(ascending = False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
TotalBath        0.597966
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

The highest correlations with sale price are as follows:

- OverallQual      0.790982
- GrLivArea        0.708624 (Above grade (ground) living area square feet)
- GarageCars       0.640409
- GarageArea       0.623431
- TotalBsmtSF      0.613581 (Total square feet of basement area)
- 1stFlrSF         0.605852 (First Floor square feet)
- TotalBath        0.597966
- FullBath         0.560664
- TotRmsAbvGrd     0.533723
- YearBuilt        0.522897
- YearRemodAdd     0.507101

I took those values which were not in my original subset and added them to it, in the form of a new subset (subset2).

In [40]:
houses_subset2 = houses_copy[['SalePrice', 'OverallQual', 'GrLivArea', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF', 'BedroomAbvGr', 'TotalBath', 'TotRmsAbvGrd', 'GarageCars']]


In [41]:
houses_subset2.head()

,SalePrice,OverallQual,GrLivArea,YearRemodAdd,TotalBsmtSF,1stFlrSF,BedroomAbvGr,TotalBath,TotRmsAbvGrd,GarageCars
0,208500,7,1710,2003,856,856,3,2.5,8,2
1,181500,6,1262,1976,1262,1262,3,2.0,6,2
2,223500,7,1786,2002,920,920,3,2.5,6,2
3,140000,7,1717,1970,756,961,3,1.0,7,3
4,250000,8,2198,2000,1145,1145,4,2.5,9,3


## Export to CSV

In [42]:
houses_subset2.to_csv('houses_subset2.csv', sep=',', index=False)